<a href="https://colab.research.google.com/github/bhagwanp367/Cardiovascular-Disease-Classification/blob/main/Cardiovascular_Disease_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Cardio Vascular Disease Classification/cardio_train.csv')

In [ ]:
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
data['Age_new']=data['age']/365

In [ ]:
data.drop(['id','age'],axis=1,inplace=True)

In [ ]:
Gender=pd.get_dummies(data['gender'],drop_first=True)
Gender

In [ ]:
Gender=pd.get_dummies(data['gender'],drop_first=True)
Cholesterol=pd.get_dummies(data['cholesterol'],drop_first=True,columns=True)
Glucose=pd.get_dummies(data['gluc'],drop_first=True)
Smoke=pd.get_dummies(data['smoke'],drop_first=True)
Alcohol=pd.get_dummies(data['alco'],drop_first=True)
Acitive=pd.get_dummies(data['active'],drop_first=True)

In [ ]:
data.drop(['gender','cholesterol','gluc','smoke','alco','active'],axis=1,inplace=True)

In [ ]:
data1 = pd.concat([data,Gender,Cholesterol,Glucose,Smoke,Alcohol,Acitive],axis=1)

In [ ]:
data1.head()

In [ ]:
X=data1.drop(['cardio'],axis=1)
y=data1['cardio']

In [ ]:
y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model1 = LogisticRegression()

In [ ]:
model1.fit(X_train,y_train)

In [ ]:
y_pred=model1.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support,confusion_matrix

In [ ]:
con_mat=confusion_matrix(y_test,y_pred)
con_mat

In [ ]:
acc_model1=accuracy_score(y_test,y_pred)
acc_model1

In [ ]:
precision_recall_fscore_support(y_test,y_pred)

### Naive bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
model2=GaussianNB()
model2.fit(X_train,y_train)

In [ ]:
y_pred1 = model2.predict(X_test)

In [ ]:
acc_model2=accuracy_score(y_test,y_pred1)

In [ ]:
acc_model2

In [ ]:
confusion_matrix(y_test,y_pred1)

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model3 = DecisionTreeClassifier()

In [ ]:
model3.fit(X_train,y_train)

In [ ]:
y_pred3 = model3.predict(X_test)

In [ ]:
acc_model3=accuracy_score(y_test,y_pred3)
acc_model3

In [ ]:
confusion_matrix(y_test,y_pred3)

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model4 = RandomForestClassifier(n_estimators=500)

In [ ]:
model4.fit(X_train,y_train)

In [ ]:
y_pred4=model4.predict(X_test)

In [ ]:
acc_model4=accuracy_score(y_test,y_pred4)

In [ ]:
acc_model4

### Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model5 = GradientBoostingClassifier(n_estimators=500)

In [ ]:
model5.fit(X_train,y_train)

In [ ]:
y_pred5=model5.predict(X_test)

In [ ]:
acc_model5=accuracy_score(y_test,y_pred5)
acc_model5

### Logistic regression with CV

> Indented block



In [ ]:
from sklearn.linear_model import LogisticRegressionCV
model6=LogisticRegressionCV()

In [ ]:
model6.fit(X_train,y_train)

In [ ]:
y_pred6=model5.predict(X_test)

In [ ]:
acc_model6=accuracy_score(y_test,y_pred6)
acc_model6

### ANN for classification

In [ ]:
import tensorflow
import keras
from tensorflow import keras
from tensorflow.keras import layers
#!pip install -U keras-tuner
from kerastuner.tuners import RandomSearch

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=2,
    executions_per_trial=2,
    directory='project1',
    project_name='Cardiovascular disease classification')

In [ ]:
tuner.search(X_train, y_train,
             epochs=2,
             validation_data=(X_test, y_test))